# <center> Panongbene Jean Mohamed Sawadogo </center>#

* $\color{red}{Course}$        : Visual Analytics, CentraleSupelec
* $\color{red}{Professor}$     : Petra Isenberg, Wesley Willett
* $\color{red}{Assignement}$   : second Assignement
* $\color{red}{Author}$        : Panongbene Jean Mohamed Sawadogo
* $\color{red}{Title}$         : Collecting & Cleaning Data

$\color{red}{Project - description : }$  In this project, we are scraping the site https://www.populationdata.net/pays/ to extract all informations relating to the countries of the world at the end, we will build a csv containing the countries with as much information as possible about each country.





# We import the necessary libraries

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests 

# All Extras Parameters

In [2]:
link_web_page = 'https://www.populationdata.net/pays/'

# the real name in the web page of the dataframe columns
array_column = ['Nom officiel','Nom propre','Continent','Sous-continent','Population',
                'Croissance démographique','Superficie','Densité','PIB','PIB/habitant',
                'Croissance du PIB','Espérance de vie',
                'Taux de natalité','Indice de fécondité','Taux de mortalité','Taux de mortalité infantile(‰ (2019))',
                'Taux d\'alphabétisation','Langues officielles','Monnaie','IDH','IPE','Nature de l"État','Chef de l"État',
                'Fête nationale','Codes ISO','Gentilé','Touristes']

# the names of all datafram columns
datafram_column = ['Nom officiel','Nom propre','Continent','Sous-continent','Population(habitants (2020))',
                  'Croissance démographique(% / an)','Superficie(km²)','Densité(habitants / km²)',
                  'PIB(milliards $USD (2017))','PIB/habitant','Croissance du PIB(% / an (2018))',
                  'Espérance de vie(ans (2019))','Taux de natalité(‰ (2019))','Indice de fécondité(enfants / femme (2019))',
                  'Taux de mortalité(‰ (2019))','Taux de mortalité infantile(‰ (2019))','Taux d"alphabétisation(% (2020))',
                   'Langues officielles','Monnaie','IDH(/ 1 (2018))','IPE','Nature de l"État','Chef de l"État','Fête nationale',
                  'Codes ISO','Gentilé','Touristes']

# all the characters we use to clean the scratched data
array_used_clean = [" milliards $USD (2017)","milliards $USD"," enfants / femme (2019)"," / 1 (2018)"," % (2020)"," ‰ (2019)"," habitants (2018)"," % / an"," km²"," habitants / km²",
                    " $USD (2017)"," % / an (2017)"," ans (2018)"," habitants /"," personnes (2017)",
                    " personnes"," $USD"," habitants"," ("," enfants / femme"," ‰", " %", " ans"]

In [3]:
def add_new_line_data_fram(name, dictionnary_data_new,datafram_country_data):
    """Use this function to add a new row in our data frame. The new row values ​​are
    the values ​​in dictionnary_data_new"""
    
    # We create the new line in data Fram
    datafram_country_data.loc[name] = 'NAN'
    
    # We put the values in our new line
    j = 0
    for i in dictionnary_data_new:
        datafram_country_data.loc[name][j] = dictionnary_data_new[i]
        j+=1
        

def clean_the_data_frame_data(values_string,array_used_clean):
    """We use this function to clean the values ​​that we are going to put in the dataframe """
    for i in array_used_clean:
        kkk = values_string.split(i)
        values_string = kkk[0]
    
    #We remove all values ; because ; is interpreted by csv as a column separator
    values_string = values_string.replace(";","")
    
    return values_string.replace("n.c.","NaN")

def save_data_frame_csv(file_name, dataframe_data):
    """We use this function to save dataframe in the csv file"""
    my_file_name = file_name + '.csv'
    dataframe_data.to_csv(my_file_name)
    print("Saved " +my_file_name)

## Function to Scrape and analyze data


In [4]:
#function 
def fetch_scrapping_pages():
    """We use this function to scrapes all informations about each country. 
     We use also this function for cleaning the data in the datafram. At the 
     end this function return a panda dataframe which contains all cleaning informations 
     of each countries. """
    
    # page Request
    page_country_first = requests.get(link_web_page, allow_redirects=True)
    
    #we process the request
    if not page_country_first.status_code == 200:
        print(" ERROR Status code: " + str(page_country_first.status_code))
        return
    else:
        soup = BeautifulSoup(page_country_first.content)
        
        # we create the data frame
        datafram_country_data = pd.DataFrame(columns=datafram_column)
    
        for tablecountry in soup.findAll('table'):
            for onecountry in tablecountry.findAll('tr'):
                # name country
                name_country = ''
                # we create the dictionnary
                dictionnary_data = {}
                
                if('data-iso-code' in onecountry.attrs):
                    for country in onecountry.findAll('a'):
                        if('pays' in country['href']):
                            link_country = country['href']
                            name_country = country.text
                            
                            # page Request
                            page_country = requests.get(link_country, allow_redirects=True)
                            #we process the request
                            if not page_country.status_code == 200:
                                print(" ERROR Status code: " + str(page_country.status_code))
                                continue
                            else:
                                soup_page_country = BeautifulSoup(page_country.content)
                                
                                for tablecountry in soup_page_country.findAll('tbody'):
                                    for table_infos in tablecountry.findAll('tr'):
                                        keys_values = []
                                        for information_line in table_infos.findAll('td'):
                                            keys_values.append(information_line.text)
                                       
                                        keys_values[0] = clean_the_data_frame_data(keys_values[0],array_used_clean)
                                        keys_values[1] = clean_the_data_frame_data(keys_values[1],array_used_clean)
                                        dictionnary_data[keys_values[0]] = keys_values[1]
                                    #we use break here because some page have a lot of tbody
                                    break
                                # we add the new ligne in the datafram
                                add_new_line_data_fram(name_country,dictionnary_data,datafram_country_data)
    return datafram_country_data

In [5]:
datafram_country_data = fetch_scrapping_pages()
datafram_country_data.head()

,Nom officiel,Nom propre,Continent,Sous-continent,Population(habitants (2020)),Croissance démographique(% / an),Superficie(km²),Densité(habitants / km²),PIB(milliards $USD (2017)),PIB/habitant,...,Langues officielles,Monnaie,IDH(/ 1 (2018)),IPE,"Nature de l""État","Chef de l""État",Fête nationale,Codes ISO,Gentilé,Touristes
Abkhazie,République d'Abkhazie,Аҧсны Аҳәынҭқарра,Asie,Asie de l'Ouest,246 313,"0,49",8 653,"28,47","0,336",1 372,...,"Abkhaze, géorgien, russe","Apsar abkhaze, Rouble russe",NaN,NaN,République,Président Raul Khadjimba,23 juillet,",","Abkhaze, Abkhazienne, Abkhazien",1 000 000
Açores,Région autonome des Açores,Região Autónoma dos Açores,Europe,Union Européenne,243 862,"-0,39",2 322,"105,02","4,220",17 398,...,Portugais,Euro,NaN,NaN,République,Président Marcelo Rebelo de Sousa,10 juin,",","Açoréenne, Açoréen, Portugaise, Portugais",1 487 428
Afghanistan,République islamique d'Afghanistan,د أفغانستان اسلامي جمهوریت,Asie,Asie centrale,34 233 212,"6,23",652 864,"52,44","19,363",521,...,Dari,Afghani,"0,496","37,74",République islamique,Président Ashraf Ghani,19 août,"AF, AFG",Afghan,20 000
Afrique du Sud,République d'Afrique du Sud,Republiek van Suid-Afrika,Afrique,Afrique Australe,57 725 606,"2,13",1 219 912,"47,32","366,298",6 340,...,"Zoulou, afrikaans, anglais",Rand,"0,705","44,73","République unitaire parlementaire, régime prés...",Président Cyril Ramaphosa,27 avril,"ZA, ZAF","Sud-Africaine, Sud-Africain",10 285 000
Albanie,République d'Albanie,Republika e Shqipërisë,Europe,Europe,2 870 324,"-0,22",28 748,"99,84","15,059",5 254,...,Albanais,Lek,"0,791","65,46",République,Président Ilir Meta,28 novembre,"AL, ALB","Albanaise, Albanais",5 375 174


In [6]:
# we save the data int the local repertory
save_data_frame_csv('country_Information_scrapping', datafram_country_data)

Saved country_Information_scrapping.csv
